In [69]:
import pandas as pd

In [70]:
papers = pd.read_csv('Papers.csv')
people = pd.read_csv('People.csv')

In [71]:
# people = people.loc[people['Birth Country'].isin(['America', 'England', 'France', 'Germany', 'Ireland', 'Scotland'])]

In [72]:
people_sampled = people.sample(len(people.index))
# benjamin = people[people['Name']=='Benjamin Franklin']
# people_sampled = pd.concat([people_sampled, benjamin])
# people_sampled = people_sampled.drop(columns=['Kin?', 'Birth Country', 'BirthPlace', 'Smallest Birth Place', 'Birth State'])
# people_sampled = people_sampled.dropna()
# people_sampled.BirthYear = people_sampled.BirthYear.astype(int)
# people_sampled.DeathYear = people_sampled.DeathYear.astype(int)

people_sampled = people_sampled[people_sampled['Name'].str.contains('\?')==False]
people_sampled.count()

Name                    744
Gender                  744
Kin?                    742
BirthYear               454
DeathYear               500
BirthPlace              561
Smallest Birth Place    395
Birth State             392
Birth Country           560
dtype: int64

In [73]:
names = people_sampled['Name'].tolist()

In [74]:
paper_sampled = papers.sample(len(papers.index))

paper_sampled = paper_sampled[paper_sampled['Author'].isin(names) & paper_sampled['Recipient'].isin(names)]

# select when Bejamin Franklin is in America
paper_sampled = paper_sampled[((paper_sampled['Author']=='Benjamin Franklin') & (paper_sampled['SourceCity']=='London'))\
                              & paper_sampled['DestinationCity'].notnull()]#((paper_sampled['Recipient']=='Benjamin Franklin') & (paper_sampled['DestinationCountry']=='America'))]
# paper_sampled = paper_sampled.sample(100)
# paper_sampled = paper_sampled[['DocumentID', 'DocumentTitle', 'Letter Year', 'Author', 'Recipient']]
# filt
# paper_sampled.count()

In [75]:
cities = paper_sampled['DestinationCity'].value_counts().index.tolist()[:5]
paper_sampled = paper_sampled[paper_sampled['DestinationCity'].isin(cities)]

In [76]:
paper_sampled

,DocumentID,DocumentTitle,DocumentURL,Letter Date,Letter Year,PrimaryLanguage,AdditionalLanguage,Author,Recipient,SourceSmallest,SourceCity,SourceState,SourceCountry,Source,DestinationSmallest,DestinationCity,DestinationState,DestinationCountry,Destination
1525,625069,To William Franklin,http://franklinpapers.org/franklin/framedVolum...,1767-11-25,1767,English,NaN,Benjamin Franklin,William Franklin,London,London,Greater London,England,"London, Greater London, England",Burlington,Burlington,New Jersey,America,"Burlington, New Jersey, America"
3020,626582,To Richard Bache,http://franklinpapers.org/franklin//yale?vol=2...,1774-9-10,1774,English,NaN,Benjamin Franklin,Richard Bache,London,London,Greater London,England,"London, Greater London, England",Philadelphia,Philadelphia,Pennsylvania,America,"Philadelphia, Pennsylvania, America"
2198,625742,To William Smith,http://franklinpapers.org/franklin//yale?vol=1...,1771-7-4,1771,English,NaN,Benjamin Franklin,William Smith,London,London,Greater London,England,"London, Greater London, England",Philadelphia,Philadelphia,Pennsylvania,America,"Philadelphia, Pennsylvania, America"
851,624395,To Mary Stevenson,http://franklinpapers.org/franklin/framedVolum...,1764-12-15,1764,English,NaN,Benjamin Franklin,Mary Stevenson Hewson,London,London,Greater London,England,"London, Greater London, England",Bromley,London,Greater London,England,"London, Greater London, England"
1302,624846,To Deborah Franklin,http://franklinpapers.org/franklin/framedVolum...,1766-12-15,1766,English,NaN,Benjamin Franklin,Deborah Franklin,London,London,Greater London,England,"London, Greater London, England",Philadelphia,Philadelphia,Pennsylvania,America,"Philadelphia, Pennsylvania, America"
3135,626697,To the New Jersey Assembly Committees of Corre...,http://franklinpapers.org/franklin//yale?vol=2...,1775-2-14,1775,English,NaN,Benjamin Franklin,New Jersey Assembly Committee of Correspondence,London,London,Greater London,England,"London, Greater London, England",Burlington,Burlington,New Jersey,America,"Burlington, New Jersey, America"
2463,626007,To Samuel Rhoads,http://franklinpapers.org/franklin//yale?vol=1...,1772-8-22,1772,English,NaN,Benjamin Franklin,"Samuel Rhoads, Sr.",London,London,Greater London,England,"London, Greater London, England",Philadelphia,Philadelphia,Pennsylvania,America,"Philadelphia, Pennsylvania, America"
288,623832,To Jane Mecom,http://franklinpapers.org/franklin/framedVolum...,1760-1-9,1760,English,NaN,Benjamin Franklin,Jane Mecom,London,London,Greater London,England,"London, Greater London, England",Boston,Boston,Massachusetts,America,"Boston, Massachusetts, America"
1711,625255,To O[liver] N[eave],http://franklinpapers.org/franklin//yale?vol=1...,1768,1768,English,NaN,Benjamin Franklin,Oliver Neave,London,London,Greater London,England,"London, Greater London, England",London,London,Greater London,England,"London, Greater London, England"
2041,625585,To Jean-Baptiste LeRoy,http://franklinpapers.org/franklin/framedVolum...,1770-10-2,1770,English,NaN,Benjamin Franklin,Jean-Baptiste LeRoy,London,London,Greater London,England,"London, Greater London, England",Paris,Paris,Île-de-France,France,"Paris, Île-de-France, France"


In [77]:
# paper_sampled['type'] = paper_sampled.apply(lambda row: 'From' if row['Author']=='Benjamin Franklin' else 'To',axis=1)

In [78]:
#  paper_sampled.groupby(['Author', 'Recipient', 'DestinationCountry'])['DestinationCountry'].transform('count')

In [79]:
recipients = paper_sampled['Recipient'].astype('category').cat.categories.tolist()
people_sampled = people_sampled[people_sampled['Name'].isin(recipients)]
benjamin = people[people['Name']=='Benjamin Franklin']
people_sampled = pd.concat([people_sampled, benjamin])

In [80]:
data = { 'nodes':[], 'links':[]}

In [81]:

import math
def gender(d):
    if d=='M':
        return 'Male'
    if d=='F':
        return 'Female'
    if d=='G':
        return 'Group'
    return 'Unknown'
def addNode(row):
    node = {
         'id':row['Name'],\
        'label':row['Name'],\
        'type':gender(row['Gender']),\
    }
    if (math.isnan(row['BirthYear'])==False):
        node['start'] = int(row['BirthYear'])
    
    if (math.isnan(row['DeathYear'])==False):
        node['end'] = int(row['DeathYear'])
    
    data['nodes'].append(node);
    
people_sampled.apply(addNode, axis=1);

In [82]:
import math

paper_grouped = paper_sampled.groupby(['Author', 'Recipient', 'DestinationCity'])
i = 0
total = 0
for name, group in paper_grouped:
    recipient = name[1]
    total = total + len(group)
    for node in data['nodes']:
        if (node['label']==recipient):
            node['weight'] = math.log(len(group), 10)
            print (i,recipient)
    i=i+1
    print (name, math.log(len(group), 10))
    
print ('total', math.log(total, 10))
for node in data['nodes']:
    if node['label'] == 'Benjamin Franklin':
        node['weight'] = math.log(total, 10)



0 Abel James
('Benjamin Franklin', 'Abel James', 'Philadelphia') 0.30102999566398114
1 Académie Royale des Sciences
('Benjamin Franklin', 'Académie Royale des Sciences', 'Paris') 0.0
2 Adolf Benzelius
('Benjamin Franklin', 'Adolf Benzelius', 'London') 0.0
3 Alexander Small
('Benjamin Franklin', 'Alexander Small', 'London') 0.0
4 Anthony Benezet
('Benjamin Franklin', 'Anthony Benezet', 'Philadelphia') 0.47712125471966244
5 Anthony Todd
('Benjamin Franklin', 'Anthony Todd', 'London') 0.47712125471966244
6 Arthur Lee
('Benjamin Franklin', 'Arthur Lee', 'London') 0.0
7 Baynton, Wharton & Morgan
('Benjamin Franklin', 'Baynton, Wharton & Morgan', 'Philadelphia') 0.30102999566398114
8 Benjamin Rush
('Benjamin Franklin', 'Benjamin Rush', 'Philadelphia') 0.7781512503836435
9 Browns & Collinson
('Benjamin Franklin', 'Browns & Collinson', 'London') 0.0
10 Cadwalader Evans
('Benjamin Franklin', 'Cadwalader Evans', 'Philadelphia') 1.2552725051033058
11 Charles Moore
('Benjamin Franklin', 'Charles M

In [83]:
for name, group in paper_grouped:       
    data['links'].append({\
                          'start':1757,\
                          'end':1775,\
                          'weight':len(group),\
                          'source':name[0],\
                          'target':name[1],\
                          'type':name[2]\
                         })

# paper_sampled.apply(lambda row: data['links'].append({\
#                                                        'id':row['DocumentID'],\
#                                                        'label':row['DocumentTitle'],\
#                                                        'start':row['Letter Year'],\
#                                                        'end':row['Letter Year'],\
#                                                        'source':row['Author'],\
#                                                        'target':row['Recipient'],\
#                                                        'type':row['DestinationCity']\
#                                                        }), axis=1);

In [84]:
import json
with open('Franklin.json', 'w') as outfile:
    json.dump(data, outfile)